In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from ipywidgets import *
%matplotlib inline

# Necessary commands to install and config jupyter dashboards
#### pip install jupyter_dashboards
#### jupyter dashboards quick-setup --sys-prefix

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Load full database
df = pickle.load(open("../data/dataset_consolidated.p", "rb"))

# Load metadata
meta = pickle.load(open("../data/metadata.p", "rb"))

In [4]:
def dateplot(x, y, **kwargs):
    ax = plt.gca()
    data = kwargs.pop("data")
    data.plot(x=x, y=y, ax=ax, grid=False, **kwargs)
    
def plot_facedgrid_by_trial_type(trial_type, subject=1, location='waist', measure='acceleration'):

    body_location_columns = list(meta[(meta.body_location==location) & (meta.measure == measure)].index)

    df_graph = df[(df.trial_type == trial_type) &
       (df.subject == subject)]
    
    df_body = df_graph[['time_seconds','trial_subtype', 'trial_num']+body_location_columns]

    df_body_X = df_graph[['time_seconds','trial_subtype', 'trial_num']+[body_location_columns[0]]]
    df_body_X.rename(columns={body_location_columns[0] : str(location + ' ' + measure)},inplace=True)
    df_body_X['axis'] = 'x'

    df_body_Y = df_graph[['time_seconds','trial_subtype', 'trial_num']+[body_location_columns[1]]]
    df_body_Y.rename(columns={body_location_columns[1] : str(location + ' ' + measure)},inplace=True)
    df_body_Y['axis'] = 'y'

    df_body_Z = df_graph[['time_seconds','trial_subtype', 'trial_num']+[body_location_columns[2]]]
    df_body_Z.rename(columns={body_location_columns[2] : str(location + ' ' + measure)},inplace=True)
    df_body_Z['axis'] = 'z'

    df_body = pd.concat([df_body_X, df_body_Y, df_body_Z])
    
    df_body = df_body[['time_seconds']+[str(location + ' ' + measure)]+['trial_subtype', 'trial_num', 'axis']]
    df_body.sort_values(['trial_num'],inplace=True)

    g = sns.FacetGrid(df_body.sort_values(['time_seconds','trial_num']), col="trial_num", row='trial_subtype', hue='axis',size=3, aspect=1.5) 
    g = g.map_dataframe(dateplot, "time_seconds", str(location + ' ' + measure))

# Complete list of trials per Type

In [5]:
@interact(type=['ADLs','Near_Falls','Falls'],
          subject=list(range(1,11)), 
          location=['waist','head','sternum','l.thigh','r.thigh','l.ankle','r.ankle'],
          measure=['acceleration', 'angular velocity', 'magnetic field'],
         )    
def plot(type,location,subject,measure):
    _ = plot_facedgrid_by_trial_type(trial_type=type, subject=subject, location=location, measure=measure)

interactive(children=(Dropdown(description='type', options=('ADLs', 'Near_Falls', 'Falls'), value='ADLs'), Dropdown(description='location', options=('waist', 'head', 'sternum', 'l.thigh', 'r.thigh', 'l.ankle', 'r.ankle'), value='waist'), Dropdown(description='subject', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=1), Dropdown(description='measure', options=('acceleration', 'angular velocity', 'magnetic field'), value='acceleration'), Output()), _dom_classes=('widget-interact',))